In [ ]:
# Colab 환경에서 kaggle.json 파일 업로드
from google.colab import files
files.upload()

# kaggle.json 파일을 .kaggle 디렉토리로 이동
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d deewakarchakraborty/portrait-paintings

Dataset URL: https://www.kaggle.com/datasets/deewakarchakraborty/portrait-paintings
License(s): CC0-1.0
100% 223M/223M [00:00<00:00, 673MB/s] 
100% 223M/223M [00:00<00:00, 717MB/s]


In [ ]:
# 데이터셋 다운로드 및 압축 해제
!kaggle datasets download -d deewakarchakraborty/portrait-paintings
!unzip portrait-paintings.zip -d portrait_paintings

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: portrait_paintings/Images/boris-kustodiev_201.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_202.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_203.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_204.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_205.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_206.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_207.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_208.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_209.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_21.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_210.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_211.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_212.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_213.jpg  
  inflating: portrait_paintings/Images/boris-kustodiev_

## DCGAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 설정값
dataroot = "portrait_paintings"
batch_size = 64
image_size = 64
nc = 3  # 이미지 채널 수 (RGB)
nz = 100 # 생성자 입력 노이즈 벡터 크기
ngf = 64 # 생성자 특징 맵 크기
ndf = 64 # 판별자 특징 맵 크기
num_epochs = 100 # **에폭 수를 100으로 늘림**
lr = 0.0002
beta1 = 0.5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 가중치 저장 경로 설정 및 디렉토리 생성
save_dir = "/content/drive/MyDrive/Colab Notebooks/Python_colab/Web Service/생성형 AI/data/dcgan"
os.makedirs(save_dir, exist_ok=True)
best_model_path = os.path.join(save_dir, "best_dcgan.pt")

# 조기 종료(Early Stopping) 설정
patience = 10
best_loss = float('inf')
patience_counter = 0

# 데이터셋 전처리 및 로더
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# 가중치 초기화 함수
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# 생성자 모델
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, input):
        return self.main(input)

# 판별자 모델
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)

# 모델 생성 및 초기화
netG = Generator().to(device)
netD = Discriminator().to(device)

# **가중치 파일 존재 여부 확인 및 로드**
if os.path.exists(best_model_path):
    print("기존 가중치 파일을 불러와서 이어서 학습을 시작합니다.")
    checkpoint = torch.load(best_model_path)
    netG.load_state_dict(checkpoint['generator_state_dict'])
    netD.load_state_dict(checkpoint['discriminator_state_dict'])
    best_loss = checkpoint.get('best_loss', float('inf'))
else:
    netG.apply(weights_init)
    netD.apply(weights_init)

# 손실 함수 및 옵티마이저
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
fixed_noise = torch.randn(64, nz, 1, 1, device=device)
real_label, fake_label = 1., 0.

# 학습 루프
print("Starting Training Loop...")
G_losses, D_losses = [], []
for epoch in range(num_epochs):
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for i, data in enumerate(progress_bar):
        # 판별자 학습
        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, device=device)
        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        errD = errD_real + errD_fake
        optimizerD.step()

        # 생성자 학습
        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()

        # 손실값 표시
        progress_bar.set_postfix(Loss_D=errD.item(), Loss_G=errG.item())

    G_losses.append(errG.item())
    D_losses.append(errD.item())

    # **조기 종료(Early Stopping) 로직**
    current_loss = errG.item() + errD.item()
    if current_loss < best_loss:
        best_loss = current_loss
        patience_counter = 0
        print(f"최적 모델 저장: 에폭 {epoch+1}, 손실: {best_loss:.4f}")
        torch.save({
            'generator_state_dict': netG.state_dict(),
            'discriminator_state_dict': netD.state_dict(),
            'best_loss': best_loss,
        }, best_model_path)
    else:
        patience_counter += 1
        print(f"손실 개선 실패. 인내심: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("조기 종료: 손실이 더 이상 개선되지 않아 학습을 중단합니다.")
            break

# 학습 후 손실 그래프 시각화
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="G")
plt.plot(D_losses, label="D")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# 원본 vs 생성 이미지 시각화
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.axis("off")
plt.title("Real Images")
dataiter = iter(dataloader)
images = next(dataiter)[0].to(device)[:64]
plt.imshow(np.transpose(vutils.make_grid(images.cpu(), padding=2, normalize=True).cpu(), (1, 2, 0)))

plt.subplot(1, 2, 2)
plt.axis("off")
plt.title("Generated Images")
with torch.no_grad():
    fake = netG(fixed_noise).detach().cpu()
plt.imshow(np.transpose(vutils.make_grid(fake, padding=2, normalize=True).cpu(), (1, 2, 0)))
plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting Training Loop...


Epoch 1/100: 100%|██████████| 90/90 [07:03<00:00,  4.70s/it, Loss_D=0.298, Loss_G=9.54]


최적 모델 저장: 에폭 1, 손실: 9.8362


Epoch 2/100: 100%|██████████| 90/90 [06:57<00:00,  4.64s/it, Loss_D=0.574, Loss_G=4.34]


최적 모델 저장: 에폭 2, 손실: 4.9108


...

Epoch 8/100: 100%|██████████| 90/90 [06:59<00:00,  4.66s/it, Loss_D=0.233, Loss_G=4.49]


손실 개선 실패. 인내심: 4/10


Epoch 9/100: 100%|██████████| 90/90 [06:58<00:00,  4.65s/it, Loss_D=0.25, Loss_G=4.33]


손실 개선 실패. 인내심: 5/10


Epoch 10/100:  21%|██        | 19/90 [01:30<05:20,  4.51s/it, Loss_D=1.03, Loss_G=3.25]

## 콘솔값 분석

1. 손실이 비슷할 때

    생성자와 판별자가 서로 균형을 이룬다는 의미입니다. 생성자가 가짜 이미지를 만들면, 판별자는 이를 50% 확률로 진짜 또는 가짜로 판단하게 됩니다. 즉, 판별자가 더 이상 속지 않기 위해 노력하고, 생성자도 더 이상 판별자를 속일 수 없어 서로 학습의 정체기에 접어든 상태입니다. 이 상태에서 생성자는 최고 품질의 가짜 이미지를 만들고 있다고 볼 수 있습니다.

2. 생성자 손실이 급격히 낮아질 때

    생성자가 너무 쉽게 판별자를 속이는 경우입니다. 판별자가 학습을 제대로 못하고 있거나, 생성된 이미지의 다양성이 부족할 수 있습니다. 이는 **모드 붕괴(Mode Collapse)**의 전조일 수 있습니다.

3. 판별자 손실이 급격히 낮아질 때

    판별자가 생성자의 가짜 이미지를 너무 쉽게 구별하는 경우입니다. 생성자가 판별자를 속일 만큼 좋은 이미지를 만들지 못하고 있다는 의미이며, 생성자에게 전달되는 학습 신호(기울기)가 약해져 학습이 제대로 진행되지 않을 수 있습니다.